<a href="https://colab.research.google.com/github/sayak-coding/Web_Scraping_and_Text_Analysis/blob/main/Web_Scraping_and_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import the required libraries

import pandas as pd # Python package for dealing with data structures  
import numpy as np # numerical python for linear algebra
import openpyxl  # a Python library for reading and writing Excel 
import requests # Requests will allow you to send HTTP/1.1 requests using Python                           
from bs4 import BeautifulSoup # for web scraping purposes to pull the data from a html website 
from textblob import TextBlob # for processing textual data   
import textstat  # to calculate statistics from text                              
import string  # package to process standard Python strings  
import re # provides full support for regular expression                
import spacy # for Industrial strength advanced NLP in Python                 
import nltk # toolkit build for working with NLP in Python     
from nltk.tokenize import word_tokenize # import module for tokenization from nltk 
from nltk.corpus import stopwords # import module for stopwords from nltk             

In [ ]:
nltk.download('punkt') # downloading punctuations from nltk 
nltk.download('stopwords') # dowloading stopwords from nltk 
nltk.download('averaged_perceptron_tagger') # downloading average perceptron tagger from nltk 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# open the file where positive words are stored 
with open("/content/drive/MyDrive/Colab Notebooks/Blackcoffer Data Engineer Assignment/20211030 Test Assignment-20230525T172247Z-001/20211030 Test Assignment/MasterDictionary/positive-words.txt","r") as pos:
    positive_words = pos.read().split("\n") 
    
# open the file where negative words are stored 
with open("/content/drive/MyDrive/Colab Notebooks/Blackcoffer Data Engineer Assignment/20211030 Test Assignment-20230525T172247Z-001/20211030 Test Assignment/MasterDictionary/negative-words.txt","r",encoding = "ISO-8859-1") as neg:
    negative_words = neg.read().split("\n")

In [ ]:
data=[] # initialize article text with a blank list 

def Text_Analysis(): # define a function 'text_analysis'

    # fetching data from given input excel sheet 
    workbook = openpyxl.load_workbook('/content/drive/MyDrive/Colab Notebooks/Blackcoffer Data Engineer Assignment/20211030 Test Assignment-20230525T172247Z-001/20211030 Test Assignment/Input.xlsx')  
    worksheet = workbook['Sheet1']
    
    # fetching the url from sheet into url variable 
    for i in range (2 , 116): # as we have only 115 url 
        url_id = int(worksheet.cell(row=i, column=1).value)
        url = (worksheet.cell(row=i, column=2).value) # iterate on second column and ith row
        
        #We need to pass argument called Headers by passing "User-Agent" to the request to bypass the mod-security error.
        headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0 Chrome/108.0.0.0 Safari/537.36"} 
        # header = {'User-Agent': "Mozilla/5.0 (X11; Linux x86_64; rv:60.0; Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Gecko/20100101 Firefox/60.0"}    
        response = requests.get(url, headers=headers) # make a request to url
    
    # apply BeautifulSoup to fetch only html parser 
        soup = BeautifulSoup(response.content, 'html.parser') 
        
    
        # fetch title from link
        try:
          # title = soup.find('h1',class_="entry-title").text.replace('\n'," ")
          title = soup.find('h1').get_text()
        except:
          continue

        #fetch content from link and remove unwanted header & punctuation
        article = soup.findAll(attrs={'class':'td-post-content'})    
        article = article[0].text.replace('\n'," ")    
        article = article.translate(str.maketrans('', '', string.punctuation)) 
    
        #tokenize the data 
        text_tokens = word_tokenize(article)
           
        #remove stopwords
        stop_words = stopwords.words('english')
        stop_tokens = [word for word in text_tokens if not word in stop_words]
    
        #count positive score usnig positive dictionary 
        positive_word_count = " ".join ([w for w in stop_tokens if w in positive_words])   
        positive_word_count=positive_word_count.split(" ")  
        Positive_score=len(positive_word_count)
    
        #count negative score usnig negative dictionary
        negative_word_count = " ".join ([w for w in stop_tokens if w in negative_words])    
        negative_word_count=negative_word_count.split(" ")    
        Negative_score=len(negative_word_count)
        
        #join filter data after removing stpowords  
        filter_article = ' '.join(stop_tokens)
        
        #words count 
        Word_Count=len(article)
        
        #Average_Sentence_Length count 
        Average_Sentence_Length = len(article.replace(' ',''))/len(re.split(r'[?!.]', article))
    
        #calculating fog index using textstat library
        Fog_Index=(textstat.gunning_fog(article)) # fog index
    
        #Average_Number_of_Words_Per_Sentence count
        Average_Number_of_Words_Per_Sentence = [len(l.split()) for l in re.split(r'[?!.]', article) if l.strip()]
        Average_Number_of_Words_Per_Sentence=(sum(Average_Number_of_Words_Per_Sentence)/len(Average_Number_of_Words_Per_Sentence))
    
        Word_Count=len(article) # count the words
       
        #function to calculate Complex_Words consedring word not ending from "ed" or "es"
        def Complex_Count(word):
            count = 0
            vowels = "AEIOUYaeiouy"
            if word[0] in vowels:
                count = count + 1
            for index in range(1, len(word)): 
                    if word[index] in vowels and word[index - 1] not in vowels:
                        count = count + 1
                        if word.endswith("es"or "ed"):
                            count = count - 1
            if count == 0:
                count = count + 1
            return count
        Complex_Word_Count = Complex_Count(article) # complex word 

    
        #function to calculate proper noun in article with help of tagging from nltk lib
        def Proper_Noun_Extractor(text):
            count = 0
            sentences = nltk.sent_tokenize(text)
            for sentence in sentences:
                words = nltk.word_tokenize(sentence)
                tagged = nltk.pos_tag(words)
                for (word, tag) in tagged:
                    if tag == 'PRP': # If the word is a proper noun
                        count = count + 1 
        
            return(count) 
        Personal_Pronouns=Proper_Noun_Extractor(article) # personal pronoun
    

        #function for sentiment analysis
        def Sentiment_Analysis(text):
            sentiment = TextBlob(text).sentiment
            return (sentiment.polarity)
    
        Polarity_Score = Sentiment_Analysis(article) # polarity score 
  
        def Sentiment_Analysis(text):
            sentiment = TextBlob(text).sentiment
            return (sentiment.subjectivity)
    
        Subjectivity_Score = Sentiment_Analysis(article) # subjective score
        
        
        #method to count average syllable count in words
        word=article.replace(' ','')
        syllable_count = 0
        for w in word:
            
            if(w=='a' or w=='e' or w=='i' or w=='o' or w=='y' or w=='u' or w=='A' or w=='E' or w=='I' or w=='O' or w=='U' or w=='Y'):
                syllable_count=syllable_count+1

        Syllable_Per_Word=(syllable_count/len(article.split()))
        
        # calculate average word length 
        Average_Word_Length=len(article.replace(' ',''))/len(article.split())
        
        # calculate percentage of complex word
        Percentage_of_Complex_Words = Complex_Word_Count / Word_Count * 100

        # insret the all analysis score in the blank list 
        data.insert(i,[url_id,url,Positive_score, Negative_score, Polarity_Score, Subjectivity_Score, Average_Sentence_Length, Percentage_of_Complex_Words, Fog_Index, Average_Number_of_Words_Per_Sentence , Complex_Word_Count, Word_Count,Syllable_Per_Word, Personal_Pronouns, Average_Word_Length])
        

# time to call the main function   
if __name__ == '__main__' :  
    Text_Analysis()
        


In [ ]:
# creating a data frame with the values of all the text analysis
df = pd.DataFrame(data,columns=['url_id','url','Positive_score','Negative_score','Polarity_Score','Subjectivity_Score', 'Average_Sentence_Length','Percentage_of_Complex_Words', 'Fog_Index', 'Average_Number_of_Words_Per_Sentence' , 'Complex_Word_Count', 'Word_Count', 'Syllable_Per_Word','Personal_Pronouns', 'Average_Word_Length'])
df.to_csv('Output_Data.csv') # save the data frame in a csv file 
df # show the final data frame 

,url_id,url,Positive_score,Negative_score,Polarity_Score,Subjectivity_Score,Average_Sentence_Length,Percentage_of_Complex_Words,Fog_Index,Average_Number_of_Words_Per_Sentence,Complex_Word_Count,Word_Count,Syllable_Per_Word,Personal_Pronouns,Average_Word_Length
0,37,https://insights.blackcoffer.com/ai-in-healthc...,77,33,0.136936,0.463364,10162.0,29.934761,721.92,1790.0,3579,11956,2.248045,22,5.677095
1,38,https://insights.blackcoffer.com/what-if-the-c...,65,37,0.074693,0.433596,6861.0,29.293051,567.40,1409.0,2424,8275,1.948900,51,4.869411
2,39,https://insights.blackcoffer.com/what-jobs-wil...,73,33,0.111470,0.483201,9220.0,30.580480,684.87,1700.0,3340,10922,2.187647,28,5.423529
3,40,https://insights.blackcoffer.com/will-machine-...,72,25,0.141419,0.487387,8061.0,29.613999,660.88,1650.0,2877,9715,1.951515,49,4.885455
4,41,https://insights.blackcoffer.com/will-ai-repla...,59,26,0.053178,0.509139,8927.0,29.369319,699.95,1739.0,3134,10671,2.015526,53,5.133410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,146,https://insights.blackcoffer.com/blockchain-fo...,28,27,0.035488,0.402814,4895.0,29.223351,369.76,911.0,1697,5807,2.109769,23,5.373216
107,147,https://insights.blackcoffer.com/the-future-of...,45,12,0.074727,0.389435,7979.0,29.090719,620.72,1550.0,2777,9546,2.042581,24,5.147742
108,148,https://insights.blackcoffer.com/big-data-anal...,30,43,0.040319,0.417892,5903.0,30.363456,465.90,1155.0,2147,7071,2.069264,14,5.110823
109,149,https://insights.blackcoffer.com/business-anal...,33,4,0.208748,0.546448,4105.0,30.336379,291.93,715.0,1461,4816,2.281119,9,5.741259


In [ ]:
from google.colab import files
files.download('Output_Data.csv') # download the csv file  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>